In [1]:
import sys
sys.path.append('..')

In [2]:
import geedatasets

# Landsat 5 SR

In [3]:
l5sr = geedatasets.fromShortName('L5SR')

In [4]:
print(l5sr)

ID: LANDSAT/LT05/C01/T1_SR
Short Name: L5SR
Spacecraft: LANDSAT
Number: 5
Sensor: TM
Process: SR
Start Date: 1984-04-16
End Date: 2012-05-05
Cloud Cover: CLOUD_COVER
Tier: 1
Bands: ['B1 (blue)', 'B2 (green)', 'B3 (red)', 'B4 (nir)', 'B5 (swir)', 'B6 (thermal)', 'B7 (swir2)', 'sr_atmos_opacity (atmos_opacity)', 'sr_cloud_qa (cloud_qa)', 'pixel_qa (pixel_qa)', 'radsat_qa (radsat_qa)']
Masks: ['pixel_qa', 'cloud_qa']
visualizers: ['trueColor', 'falseColor', 'NSR', 'atmos_opacity']



### Get the collection, an image and visualize

In [5]:
import ee
ee.Initialize()
import ipygee as ui

In [6]:
Map = ui.Map()
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [7]:
point = ee.Geometry.Point([-72, -42])
Map.centerObject(point, 12)

In [8]:
collection = l5sr.collection().filterBounds(point)

In [9]:
image = collection.first()

In [10]:
Map.addLayer(image, l5sr.visualization('trueColor'), 'L5 SR image trueColor')

# Get the QA method
The attribute `common_masks` is a list with the dataset's QA methods. The first item is the most used. For example, for Sentinel 2 the fist items is the QA method for masking using the SCL band, and the second is for the QA60 band.

In [11]:
qa = l5sr.masks[0]

In [12]:
print(qa)

'pixel_qa' Mask
options: ['clear', 'water', 'shadow', 'snow', 'cloud', 'high_confidence_cloud', 'high_confidence_cirrus']
default negatives: ['water', 'shadow', 'snow', 'cloud', 'high_confidence_cloud', 'high_confidence_cirrus']
default positives: ['clear']



In [13]:
masked = qa.apply(image)

In [14]:
Map.addLayer(masked, l5sr.visualization('falseColor'), 'L5 SR masked')

In [15]:
# You can get a custom mask
# For example, where is the snow?
snow = qa.get(image, positives=['snow'])
Map.addLayer(snow.selfMask(), dict(palette=['red']), name='snow (red)')

In [16]:
# The mask can also be applied in a map function
filtered = collection.limit(3).map(lambda i: qa.apply(i))

In [17]:
Map.addImageCollection(filtered, l5sr.visualization('NSR'), 'L5 SR {system_date}', datePattern='yyyy-MM-dd')

## Other bands
### Atmospheric opacity

In [18]:
atmos = l5sr.getBandByAlias('atmos_opacity')

In [19]:
print(atmos)

Band sr_atmos_opacity(atmos_opacity):
resolution: 30
precision: int16
units: None
classes: {'clear': 'value<0.1', 'average': '0.1>=value<=0.3', 'hazy': 'value>0.3'}
positives: ['clear', 'average']
negatives: ['hazy']



In [20]:
Map.addLayer(image, l5sr.visualization('atmos_opacity'), 'atmospheric opacity')

In [21]:
atmosI = atmos.decodeImage(image)

In [22]:
Map.addLayer(atmosI, name='atmospheric opacity decoded')

# Landsat 8

In [23]:
l8sr = geedatasets.fromShortName('L8SR')

In [24]:
print(l8sr)

ID: LANDSAT/LC08/C01/T1_SR
Short Name: L8SR
Spacecraft: LANDSAT
Number: 8
Sensor: OLI
Process: SR
Start Date: 2013-03-18
End Date: None
Cloud Cover: CLOUD_COVER
Tier: 1
Bands: ['B1 (coastal_aerosol)', 'B2 (blue)', 'B3 (green)', 'B4 (red)', 'B5 (nir)', 'B6 (swir)', 'B10 (thermal)', 'B11 (thermal2)', 'sr_aerosol (sr_aerosol)', 'pixel_qa (pixel_qa)', 'radsat_qa (radsat_qa)']
Masks: ['pixel_qa']
visualizers: ['trueColor', 'falseColor', 'NSR']



In [25]:
collectionL8 = l8sr.collection().filterBounds(point)

In [26]:
imageL8 = collectionL8.first()

In [27]:
Map.addLayer(imageL8, l8sr.visualization('NSR'), 'Landsat 8 SR NSR')

In [28]:
l8qa = l8sr.masks[0]

In [29]:
print(l8qa)

'pixel_qa' Mask
options: ['clear', 'water', 'shadow', 'snow', 'cloud', 'high_confidence_cloud', 'high_confidence_cirrus']
default negatives: ['water', 'shadow', 'snow', 'cloud', 'high_confidence_cloud', 'high_confidence_cirrus']
default positives: ['clear']



In [30]:
Map.addLayer(l8qa.apply(imageL8), l8sr.visualization('falseColor'), 'L8 SR masked falseColor')

In [31]:
# Mask a collection

In [32]:
maskedL8 = collectionL8.map(lambda i: l8qa.apply(i))

In [33]:
Map.addImageCollection(maskedL8.limit(3), l8sr.visualization('falseColor'), 'L8 SR masked {system_date}', datePattern='yyyy-MM-dd')

## Extra bands: NBR

In [34]:
l8sr.extraBandNames()

['NDVI', 'NBR']

In [35]:
nbr = l8sr.getBandByName('NBR')

In [36]:
nbr_col = collectionL8.map(lambda i: l8sr.addBand(i, nbr))

In [37]:
Map.addImageCollection(nbr_col.limit(3), namePattern='L8 SR NBR {system_date}')